In [1]:
#importing Libraries
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [28]:
#url is link to wikipedia page link
url='https://en.wikipedia.org/wiki/Community_areas_in_Chicago'
#Scrapping the link
res=requests.get(url)
#getting the data from responser
soup = BeautifulSoup(res.content,'lxml')
#print(soup.prettify())
#we know the table we wanted to extract is 1st table of data
table=soup.find_all('table')
table=table[1]
print(table)


<table class="wikitable">
<caption>North Side
</caption>
<tbody><tr>
<th>Number</th>
<th>Community area</th>
<th>Neighborhoods
</th></tr>
<tr>
<td style="text-align: center;">05
</td>
<td><a href="/wiki/North_Center,_Chicago" title="North Center, Chicago">North Center</a>
</td>
<td>
<ul><li><a href="/wiki/North_Center,_Chicago#Horner_Park" title="North Center, Chicago">Horner Park</a></li>
<li><a href="/wiki/North_Center,_Chicago#Roscoe_Village" title="North Center, Chicago">Roscoe Village</a></li></ul>
</td></tr>
<tr>
<td style="text-align: center;">06
</td>
<td><a href="/wiki/Lake_View,_Chicago" title="Lake View, Chicago">Lake View</a>
</td>
<td>
<ul><li><a href="/wiki/Lake_View,_Chicago#Boystown" title="Lake View, Chicago">Boystown</a></li>
<li><a href="/wiki/Lake_View,_Chicago#Lakeview_East" title="Lake View, Chicago">Lake View East</a></li>
<li><a href="/wiki/Lake_View,_Chicago#Graceland_West" title="Lake View, Chicago">Graceland West</a></li>
<li><a href="/wiki/Lake_View,_Chicago

In [29]:
#Creating the empty dataframe for getting the data for North Side Community data
df_Chicago_raw=pd.DataFrame(columns={'Neighborhood','Community area','CNumber'},index = range(0,6))
df_Chicago_raw.columns = ['Cnumber', 'Community Area','Neighborhood']
df_Chicago_raw

,Cnumber,Community Area,Neighborhood
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN


In [30]:
#Extracting the data from the table and adding it to dataframe
row_marker = 0
for row in table.find_all('tr'):
    #print(row_marker)
    column_marker = 0
    #print(column_marker)
    columns = row.find_all('td')
    #print(columns)
    for column in columns:
        if(column_marker==2):
            neighborhood=""
            for li in column.contents[1].find_all('li'):
                neighborhood=neighborhood+li.get_text()
                neighborhood=neighborhood+","
                #print(neighborhood)
            df_Chicago_raw.iat[row_marker,column_marker] = neighborhood
            #print(neighborhood)
        else:
            df_Chicago_raw.iat[row_marker,column_marker] = column.get_text().strip()
            #print(column.get_text())
            column_marker += 1
            #print(column_marker)
    row_marker += 1
    


In [31]:
#droping the rows with NA value
df_Chicago_raw.dropna(axis=0)


,Cnumber,Community Area,Neighborhood
1,05,North Center,"Horner Park,Roscoe Village,"
2,06,Lake View,"Boystown,Lake View East,Graceland West,South E..."
3,07,Lincoln Park,"Old Town Triangle,Park West,Ranch Triangle,She..."
4,21,Avondale,"Belmont Gardens,Chicago's Polish Village,Kosci..."
5,22,Logan Square,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme..."


### Adding population data from IBM data assests to new dataframe

In [32]:
# The code was removed by Watson Studio for sharing.

In [33]:
#Changing the name of columns
df_area_population.columns=['Cnumber','Zipcode','Population']
df_area_population.head()


,Cnumber,Zipcode,Population
0,1,60626,48980
1,1,60645,5820
2,1,60660,191
3,2,60626,1159
4,2,60645,39454


In [34]:
#Adding the Population by Community Area
df=df_area_population.groupby('Cnumber',as_index=False)[['Population']].sum()
type(df)
df.head()

,Cnumber,Population
0,1,54991
1,2,71942
2,3,56362
3,4,39493
4,5,31867


In [35]:
#Adding leading zeros pto 2 digits to merge the data on raw data
df['Cnumber']=df['Cnumber'].apply(lambda x: '{0:0>2}'.format(x))
df.head()

,Cnumber,Population
0,01,54991
1,02,71942
2,03,56362
3,04,39493
4,05,31867


In [36]:
#Creating a merged dataframe for Northside area with population
df_Chicago_merge=pd.merge(df_Chicago_raw,df,on='Cnumber',right_index=True)
df_Chicago_merge.head()

,Cnumber,Community Area,Neighborhood,Population
1,05,North Center,"Horner Park,Roscoe Village,",31867
2,06,Lake View,"Boystown,Lake View East,Graceland West,South E...",94368
3,07,Lincoln Park,"Old Town Triangle,Park West,Ranch Triangle,She...",64116
4,21,Avondale,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262
5,22,Logan Square,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595


In [37]:
# Getting Community area  in list form to find latitude and longitude
Area_list=df_Chicago_merge['Community Area'].tolist()
Area_list

['North Center', 'Lake View', 'Lincoln Park', 'Avondale', 'Logan Square']

In [38]:
#Creating a dictionary of Community area ,latitude longitude
dict=[]
for area in Area_list:
    address = area+",Illinois"
    print(address)
    geolocator = Nominatim(user_agent="Chicago")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of '+ area ,' are {}, {}.'.format(latitude, longitude))
    dict.append({'Community Area': area, 'Latitude': latitude, 'Longitude':
        longitude})
dict

North Center,Illinois
The geograpical coordinate of North Center  are 41.9561073, -87.6791596.
Lake View,Illinois
The geograpical coordinate of Lake View  are 41.24672625, -89.914235390328.
Lincoln Park,Illinois
The geograpical coordinate of Lincoln Park  are 41.94029765, -87.6381171054176.
Avondale,Illinois
The geograpical coordinate of Avondale  are 41.9389208, -87.711168.
Logan Square,Illinois
The geograpical coordinate of Logan Square  are 41.9283996, -87.7067641113273.


[{'Community Area': 'North Center',
  'Latitude': 41.9561073,
  'Longitude': -87.6791596},
 {'Community Area': 'Lake View',
  'Latitude': 41.24672625,
  'Longitude': -89.914235390328},
 {'Community Area': 'Lincoln Park',
  'Latitude': 41.94029765,
  'Longitude': -87.6381171054176},
 {'Community Area': 'Avondale',
  'Latitude': 41.9389208,
  'Longitude': -87.711168},
 {'Community Area': 'Logan Square',
  'Latitude': 41.9283996,
  'Longitude': -87.7067641113273}]

In [39]:
#converting Dictionary to dataFrame
df_lat_lng=pd.DataFrame(dict)
df_lat_lng

,Community Area,Latitude,Longitude
0,North Center,41.956107,-87.679160
1,Lake View,41.246726,-89.914235
2,Lincoln Park,41.940298,-87.638117
3,Avondale,41.938921,-87.711168
4,Logan Square,41.928400,-87.706764


In [40]:
#Adding latitude, longitude to chicago dataframe
df_Chicago_merge=pd.merge(df_Chicago_merge,df_lat_lng,on='Community Area',right_index=True)
df_Chicago_merge.head()

,Cnumber,Community Area,Neighborhood,Population,Latitude,Longitude
1,05,North Center,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
2,06,Lake View,"Boystown,Lake View East,Graceland West,South E...",94368,41.246726,-89.914235
3,07,Lincoln Park,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
4,21,Avondale,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
5,22,Logan Square,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.928400,-87.706764


In [41]:
#Finding the location cordinate for lincon paark
address = 'Chicago,IL'

geolocator = Nominatim(user_agent="Chicago")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


In [42]:
# The code was removed by Watson Studio for sharing.

In [71]:
LIMIT=100
radius=500
search_query = 'Gym','yoga'
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude,
            VERSION,
            search_query,
            radius, 
            LIMIT)
url

"https://api.foursquare.com/v2/venues/search?&client_id=OOBSCGYV3Y1SA5EIM0T5VDAM4OG2NEDAFZNPEXN5PEGKSWSW&client_secret=HOZPTBKCUKEJGBSFIKH2NTGTX411KEFCY40UCK14AL5NLG55&ll=41.8755616,-87.6244212&v=20180605&query=('Gym', 'yoga')&radius=500&limit=100"

In [72]:
results = requests.get(url).json()["response"]['venues']
results

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d102941735',
    'name': 'Yoga Studio',
    'pluralName': 'Yoga Studios',
    'primary': True,
    'shortName': 'Yoga Studio'}],
  'hasPerk': False,
  'id': '4d657823601d88bfcb19c3bb',
  'location': {'address': '480 South Michigan Ave.',
   'cc': 'US',
   'city': 'Chicago',
   'country': 'United States',
   'distance': 30,
   'formattedAddress': ['480 South Michigan Ave.',
    'Chicago, IL',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.875839,
     'lng': -87.624405}],
   'lat': 41.875839,
   'lng': -87.624405,
   'state': 'IL'},
  'name': 'Yoga @ Roosevelt University',
  'referralId': 'v-1550636817'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d102941735',
    'name': 'Yoga Studio',
    'pluralName

### Function to find Gym Yoga given location and creating a dataframe

In [50]:

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 500 # limit of number of venues returned by Foursquare API
        radius 
        search_query = 'Gym','yoga'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng,
            VERSION,
            search_query,
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['venues']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['location']['formattedAddress'][-2],
            v['location']['distance'],
            v['location']['lat'], 
            v['location']['lng'],
            ) for v in results])

    nearby_venues = pd.DataFrame([item for gym_venues in venues_list for item in gym_venues])
    nearby_venues.columns = ['Community Area', 
                  'Community Area Latitude', 
                  'Community Area Longitude', 
                  'Venue', 
                  'PostalCode',
                  'distance',
                  'Venue Latitude', 
                  'Venue Longitude',
                  ]      
    
    return(nearby_venues)

In [51]:
chicago_venues = getNearbyVenues(names=df_Chicago_merge['Community Area'],
                                   latitudes=df_Chicago_merge['Latitude'],
                                   longitudes=df_Chicago_merge['Longitude']
                                  )

North Center
Lake View
Lincoln Park
Avondale
Logan Square


In [52]:
print(chicago_venues.shape)
chicago_venues.head()


(148, 8)


,Community Area,Community Area Latitude,Community Area Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
0,North Center,41.956107,-87.67916,CorePower Yoga,"Chicago, IL 60657",1881,41.940761,-87.669649
1,North Center,41.956107,-87.67916,Bloom Yoga Studio,"Chicago, IL 60625",1673,41.966510,-87.693761
2,North Center,41.956107,-87.67916,CorePower Yoga,"Chicago, IL 60640",2044,41.962225,-87.655874
3,North Center,41.956107,-87.67916,That's So Raven Yoga Studio,"Chicago, IL 60613",315,41.954017,-87.676579
4,North Center,41.956107,-87.67916,Spring Yoga & Fitness,"Chicago, IL 60613",304,41.953447,-87.678305


In [53]:
#Creating a dataframe to
chicaco_gym_count=chicago_venues.groupby('Community Area',as_index=False)[['Venue']].count()
chicaco_gym_count.columns=['Community Area','Total Gyms']
chicaco_gym_count

,Community Area,Total Gyms
0,Avondale,27
1,Lake View,1
2,Lincoln Park,50
3,Logan Square,20
4,North Center,50


### Prepare for K means

In [54]:
df_Chicago_Kmeans=pd.merge(chicago_venues,df_Chicago_merge,on='Community Area',right_index=True)
df_Chicago_Kmeans.head()


,Community Area,Community Area Latitude,Community Area Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
0,North Center,41.956107,-87.67916,CorePower Yoga,"Chicago, IL 60657",1881,41.940761,-87.669649,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
1,North Center,41.956107,-87.67916,Bloom Yoga Studio,"Chicago, IL 60625",1673,41.966510,-87.693761,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
2,North Center,41.956107,-87.67916,CorePower Yoga,"Chicago, IL 60640",2044,41.962225,-87.655874,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
3,North Center,41.956107,-87.67916,That's So Raven Yoga Studio,"Chicago, IL 60613",315,41.954017,-87.676579,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
4,North Center,41.956107,-87.67916,Spring Yoga & Fitness,"Chicago, IL 60613",304,41.953447,-87.678305,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916


In [55]:
df_Chicago_Kmeans

,Community Area,Community Area Latitude,Community Area Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
0,North Center,41.956107,-87.679160,CorePower Yoga,"Chicago, IL 60657",1881,41.940761,-87.669649,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
1,North Center,41.956107,-87.679160,Bloom Yoga Studio,"Chicago, IL 60625",1673,41.966510,-87.693761,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
2,North Center,41.956107,-87.679160,CorePower Yoga,"Chicago, IL 60640",2044,41.962225,-87.655874,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
3,North Center,41.956107,-87.679160,That's So Raven Yoga Studio,"Chicago, IL 60613",315,41.954017,-87.676579,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
4,North Center,41.956107,-87.679160,Spring Yoga & Fitness,"Chicago, IL 60613",304,41.953447,-87.678305,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
5,North Center,41.956107,-87.679160,Gym,Illinois,669,41.951291,-87.674316,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
6,North Center,41.956107,-87.679160,guaranteedRate - The Gym,"Chicago, IL",485,41.953445,-87.674511,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
7,North Center,41.956107,-87.679160,Ravenswood United Church Gym,"Chicago, IL 60618",524,41.960603,-87.681071,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
8,North Center,41.956107,-87.679160,XSport Fitness,"Chicago, IL 60657",1848,41.941285,-87.669099,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
9,North Center,41.956107,-87.679160,Coconut Yoga,"Chicago, IL 60618",623,41.960786,-87.683291,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160


In [57]:
Chicago_clustering=pd.DataFrame([df_Chicago_Kmeans.Population, df_Chicago_Kmeans.distance]).transpose()
Chicago_clustering

,Population,distance
0,31867,1881
1,31867,1673
2,31867,2044
3,31867,315
4,31867,304
5,31867,669
6,31867,485
7,31867,524
8,31867,1848
9,31867,623


In [58]:
# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Chicago_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [59]:
df_Chicago_Kmeans.insert(0, 'Cluster Labels', kmeans.labels_)

In [60]:
df_Chicago_Kmeans

,Cluster Labels,Community Area,Community Area Latitude,Community Area Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
0,0,North Center,41.956107,-87.679160,CorePower Yoga,"Chicago, IL 60657",1881,41.940761,-87.669649,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
1,0,North Center,41.956107,-87.679160,Bloom Yoga Studio,"Chicago, IL 60625",1673,41.966510,-87.693761,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
2,0,North Center,41.956107,-87.679160,CorePower Yoga,"Chicago, IL 60640",2044,41.962225,-87.655874,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
3,0,North Center,41.956107,-87.679160,That's So Raven Yoga Studio,"Chicago, IL 60613",315,41.954017,-87.676579,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
4,0,North Center,41.956107,-87.679160,Spring Yoga & Fitness,"Chicago, IL 60613",304,41.953447,-87.678305,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
5,0,North Center,41.956107,-87.679160,Gym,Illinois,669,41.951291,-87.674316,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
6,0,North Center,41.956107,-87.679160,guaranteedRate - The Gym,"Chicago, IL",485,41.953445,-87.674511,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
7,0,North Center,41.956107,-87.679160,Ravenswood United Church Gym,"Chicago, IL 60618",524,41.960603,-87.681071,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
8,0,North Center,41.956107,-87.679160,XSport Fitness,"Chicago, IL 60657",1848,41.941285,-87.669099,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160
9,0,North Center,41.956107,-87.679160,Coconut Yoga,"Chicago, IL 60618",623,41.960786,-87.683291,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.679160


### Mapping of Clusters

In [62]:
# Visualizing resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Chicago_Kmeans['Venue Latitude'], df_Chicago_Kmeans['Venue Longitude'], df_Chicago_Kmeans['Community Area'], df_Chicago_Kmeans['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
df_Chicago_Kmeans.loc[df_Chicago_Kmeans['Cluster Labels'] == 0, df_Chicago_Kmeans.columns[[1] + list(range(5, df_Chicago_Kmeans.shape[1]))]]

,Community Area,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
0,North Center,"Chicago, IL 60657",1881,41.940761,-87.669649,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
1,North Center,"Chicago, IL 60625",1673,41.966510,-87.693761,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
2,North Center,"Chicago, IL 60640",2044,41.962225,-87.655874,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
3,North Center,"Chicago, IL 60613",315,41.954017,-87.676579,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
4,North Center,"Chicago, IL 60613",304,41.953447,-87.678305,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
5,North Center,Illinois,669,41.951291,-87.674316,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
6,North Center,"Chicago, IL",485,41.953445,-87.674511,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
7,North Center,"Chicago, IL 60618",524,41.960603,-87.681071,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
8,North Center,"Chicago, IL 60657",1848,41.941285,-87.669099,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916
9,North Center,"Chicago, IL 60618",623,41.960786,-87.683291,05,"Horner Park,Roscoe Village,",31867,41.956107,-87.67916


In [64]:
df_Chicago_Kmeans.loc[df_Chicago_Kmeans['Cluster Labels'] == 1, df_Chicago_Kmeans.columns[[1] + list(range(5, df_Chicago_Kmeans.shape[1]))]]

,Community Area,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
51,Lincoln Park,"Chicago, IL 60657",1440,41.934174,-87.653444,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
52,Lincoln Park,"Chicago, IL 60657",83,41.940563,-87.639066,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
53,Lincoln Park,"Chicago, IL 60657",461,41.936218,-87.639102,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
54,Lincoln Park,"Chicago, IL 60657",295,41.938877,-87.641138,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
55,Lincoln Park,"Chicago, IL 60657",189,41.938610,-87.638432,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
56,Lincoln Park,"Chicago, IL 60657",516,41.940551,-87.644350,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
57,Lincoln Park,"Chicago, IL 60657",135,41.939801,-87.639611,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
58,Lincoln Park,"Chicago, IL 60657",706,41.934083,-87.639828,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
59,Lincoln Park,"Chicago, IL",677,41.935390,-87.642965,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117
60,Lincoln Park,"Chicago, IL 60614",1092,41.930695,-87.640868,07,"Old Town Triangle,Park West,Ranch Triangle,She...",64116,41.940298,-87.638117


In [65]:
df_Chicago_Kmeans.loc[df_Chicago_Kmeans['Cluster Labels'] == 2, df_Chicago_Kmeans.columns[[1] + list(range(5, df_Chicago_Kmeans.shape[1]))]]

,Community Area,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
128,Logan Square,"Chicago, IL 60647",1707,41.930220,-87.686289,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
129,Logan Square,"Chicago, IL",894,41.923377,-87.698335,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
130,Logan Square,"Chicago, IL",366,41.925181,-87.707667,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
131,Logan Square,"Chicago, IL 60647",502,41.931398,-87.711297,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
132,Logan Square,"Chicago, IL",505,41.928788,-87.700679,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
133,Logan Square,"Chicago, IL 60618",595,41.933678,-87.705568,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
134,Logan Square,"Chicago, IL 60647",425,41.924590,-87.706386,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
135,Logan Square,"Chicago, IL 60639",574,41.931892,-87.711866,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
136,Logan Square,"Chicago, IL 60647",623,41.926438,-87.699713,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764
137,Logan Square,"Chicago, IL 60647",934,41.921242,-87.700874,22,"Belmont Gardens,Bucktown,Kosciuszko Park,Palme...",73595,41.9284,-87.706764


In [66]:
df_Chicago_Kmeans.loc[df_Chicago_Kmeans['Cluster Labels'] == 3, df_Chicago_Kmeans.columns[[1] + list(range(5, df_Chicago_Kmeans.shape[1]))]]

,Community Area,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
101,Avondale,"Chicago, IL 60618",745,41.933678,-87.705568,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
102,Avondale,"Chicago, IL 60647",837,41.931398,-87.711297,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
103,Avondale,"Chicago, IL 60639",784,41.931892,-87.711866,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
104,Avondale,"Chicago, IL 60618",2133,41.955967,-87.722942,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
105,Avondale,"Chicago, IL",2030,41.923377,-87.698335,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
106,Avondale,"Chicago, IL 60641",1942,41.947897,-87.731286,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
107,Avondale,"Chicago, IL",1500,41.947711,-87.697424,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
108,Avondale,"Chicago, IL 60618",1215,41.948899,-87.705217,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
109,Avondale,"Chicago, IL",1423,41.928788,-87.700679,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168
110,Avondale,"Chicago, IL 60618",1822,41.951607,-87.697258,21,"Belmont Gardens,Chicago's Polish Village,Kosci...",39262,41.938921,-87.711168


In [67]:
df_Chicago_Kmeans.loc[df_Chicago_Kmeans['Cluster Labels'] == 4, df_Chicago_Kmeans.columns[[1] + list(range(5, df_Chicago_Kmeans.shape[1]))]]

,Community Area,PostalCode,distance,Venue Latitude,Venue Longitude,Cnumber,Neighborhood,Population,Latitude,Longitude
50,Lake View,"Kewanee, IL 61443",871,41.253928,-89.918316,06,"Boystown,Lake View East,Graceland West,South E...",94368,41.246726,-89.914235
